# AIカメラと扉開け閉めプログラムの結合

先ほど作成した扉開け閉めプログラムと顔認識プログラムを連結しましょう。顔認識プログラムで、学習した人と判断したら、上記プログラムの扉を開けるコードを実行します。

In [ ]:
import sys
import time
from hub import port      # ポート接続用モジュールを読み込む
from spike import Motor   # モータ用モジュールを読み込む


# HuskyLens用モジュールを読み込む
sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

# setup serial port
port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

# initialize HusyLens
husky = HuskyLens(port.F)
husky.send_CMD_REQ_ALGO(Algo.FACE_RECOGNITION)


# モーターを初期化します。
motorA = Motor('A')
motorB = Motor('B')


while True:

    # read target info from HuskyLens
    blocks=husky.read_blocks()
    if len(blocks) > 0:
        print('!! find target(face) !!')
        print(blocks)
        type, id, x, y, w, h = blocks[0]
        print(type, id, x, y, w, h)
        if id == 1:
            print('learned face')
            motorA.start(speed=-3)
            motorB.start(speed=3)
            time.sleep(3)
            motorA.stop()
            motorB.stop()
            time.sleep(2)
        else:
            print('unkown face')
        time.sleep(0.5)
    else:
        print('no target(face)')
        # close door
        motorA.run_to_position(0,speed=5)
        motorB.run_to_position(0,speed=5)

プログラムとしてはこれで完成ですが、もう少し効果的に扉を開けたいので、扉を開ける時の効果音を再生させます。プログラムは以下です